# Práctica 7 - Parte 2: GMM como Proceso Generativo


### Versión 1.0


### Nombre:


### Fecha:



Librerías que emplearemos en la práctica (puedes importar más si lo consideras necesario):


In [3]:
from math import pi

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

# descomenta %matplotlib qt si prefieres plots interactivos
# %matplotlib qt

Funciones auxiliares para visualizar resultados en 3D usando [`plotly`](https://plotly.com/python/):

In [4]:
def visualize_subset_in_grid(
    images,
    subset=25,
    random=False,
    ncols=None,
    figsize=None,
    fig=None,
    ax=None,
):
    """Visualiza un subset de imágenes en una cuadrícula.

    Args:
        images: (n, 784) array con n imágenes 28x28 de Fasion-MNIST.
        subset: Si es un integer, número de imágenes a visualizar, si es una secuencia
            de enteros, visualiza las imágenes con esos índices.
        random: Si True y subset es un entero, las imágenes se seleccionarán
            aleatoriamente.
        ncols: Número de columnas de la cuadrícula. Si es None, se usará una grid
            cuadrada.
        figsize: (w, h) Tamaño de la figura.
        fig: Figura de matplotlib en la que visualizar las imágenes. Si es None, se
            creará una nueva figura.
        ax: Array de ejes de matplotlib en los que visualizar las imágenes. Si es None,
            se crearán nuevos ejes.
    """
    assert images.shape[1] == 784, "Las imágenes deben ser de 28x28 píxeles."

    if isinstance(subset, int):
        subset = min(subset, images.shape[0])
        if random:
            subset = np.random.choice(images.shape[0], subset, replace=False)
        else:
            subset = np.arange(subset)

    n = len(subset)
    if ncols is None:
        ncols = int(np.ceil(np.sqrt(n)))
    nrows = int(np.ceil(n / ncols))

    if fig is None or ax is None:
        fig, ax = plt.subplots(nrows, ncols, figsize=figsize)
    for i, axi in zip(subset, ax.ravel()):
        axi.imshow(images[i].reshape(28, 28), cmap="gray")
        # axi.axis("off")

    for axi in ax.ravel():
        axi.set(xticks=[], yticks=[])

    fig.tight_layout()
    return fig, ax


def plotly_3d_points(fig, points, row=None, col=None, marker_cfg=None):
    """Plot de puntos 3D con plotly

    Args:
        fig: figura de plotly
        points: (n, 3) array con los puntos RGB
    """
    # assert points.shape[1] == 3, "Los puntos deben tener shape (n, 3)"

    # define and update default marker properties
    m_cfg = dict(
        color=points / 255,
        size=3.0,
        opacity=1.0,
        symbol="square",
        line_color="white",
        line_width=0.1,
    )
    m_cfg = {**m_cfg, **(marker_cfg or {})}

    fig.add_trace(
        go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode="markers",
            marker=m_cfg,
            # name="3D points",
            showlegend=False,
        ),
        row=row,
        col=col,
    )


def plotly_kmeans_results_3d(fig, kmeans, X, row=None, col=None, c_per_cluster=None):
    """Visualización de los resultados de KMeans con datos 3D"""
    means = kmeans.cluster_centers_
    if c_per_cluster is None:
        # c_per_cluster = means / 255
        c_per_cluster = (means - means.min(axis=0)) / np.ptp(means, axis=0)

    labels = kmeans.predict(X)
    for i, (mean, color) in enumerate(zip(means, c_per_cluster)):
        X_cluster = X[labels == i]
        m_cfg_point = {
            "color": np.repeat(color[None], X_cluster.shape[0], axis=0),
            "size": 0.5,
            # "opacity": 0.1,
            "line_width": None,
        }
        m_cfg_means = {"size": 8.0, "symbol": "circle"}
        plotly_3d_points(fig, X_cluster, row=row, col=col, marker_cfg=m_cfg_point)
        plotly_3d_points(fig, mean[None], row=row, col=col, marker_cfg=m_cfg_means)


def plotly_add_ellipsoids(
    fig, covs, centers, colors, std_factors=(1, 2, 3), row=None, col=None
):
    """Muestra elipsoides 3D con plotly

    Args:
        fig: figura plotly.
        covs: (ncovs, 3, 3) matrices de covarianza para cada elipsoide.
        centers: (ncovs, 3) centros de los elipsoides.
        colors: (ncovs, 3) colores para cada elipsoide en RGB.
        std_factor: factor para controlar el tamaño de los elipsoides.
    """
    assert covs.shape[-2:] == (3, 3) and covs.ndim == 3
    assert centers.shape[-1] == 3 and centers.ndim == 2
    std_factors = np.sort(std_factors)
    alphas = np.linspace(0.6, 0.1, len(std_factors))

    # sampling de puntos en la esfera unitaria
    n = 20  # controla la resolución de la malla esférica
    # longitud (theta) y latitud (phi)
    theta = np.linspace(-pi, pi, n)
    phi = np.linspace(-0.5 * pi, 0.5 * pi, n)
    theta, phi = np.meshgrid(theta, phi)
    # coordenadas Cartesianas
    x = np.cos(phi) * np.cos(theta)
    y = np.cos(phi) * np.sin(theta)
    z = np.sin(phi)
    sphere = np.stack((x, y, z), axis=0)
    sphere_r = sphere.reshape(3, -1)

    # puntos en esfera unitaria -> puntos en elipsoides correspondientes a las covs
    vars_, evecs = np.linalg.eigh(covs)
    stds = np.sqrt(vars_)
    ellipsoids = evecs @ (stds[..., None] * sphere_r)  # (ncovs, 3, n*n)
    ellipsoids = ellipsoids.reshape((covs.shape[0], *sphere.shape))  # (ncovs, 3, n, n)

    for std_f, alpha in zip(std_factors, alphas):
        # scale and add offset (centers)
        ellipsoids_ = std_f * ellipsoids + centers[:, :, None, None]

        for ell, color in zip(ellipsoids_, colors):
            color = (color * 255).astype(int).clip(0, 255)
            data = go.Mesh3d(
                x=ell[0].ravel(),
                y=ell[1].ravel(),
                z=ell[2].ravel(),
                alphahull=0,
                opacity=alpha,
                color=f"rgb({color[0]}, {color[1]}, {color[2]})",
            )
            fig.add_trace(data, row=row, col=col)


def plotly_gmm_results_3d(
    fig,
    X,
    gmm,
    pca=None,
    c_per_component=None,
    show_covs=True,
    std_factors=(1,),
    pca3_layout=True,
    means_to_show=8,
    row=None,
    col=None,
):
    """Plot de los resultados de GMM en 3D

    Esta función, emplea plotly para:
    1) mostrar en 3D los puntos correspondientes a las 3 primeras dimensiones de los
        datos de entrada X.
    2) mostrar, como elipsoides, las distribuciones Gaussianas (sus 3 primeras dimensiones)
        ajustadas por el modelo GMM.
    3) opcionalmente, visualizar como imágenes las medias de las componentes Gaussianas.
        Si estas medias están expresadas en un espacio de dimenionalidad reducidad
        (transformadas con PCA), el correspondiente objeto de PCA debe ser proporcionado
        en el argumento pca.

    Args:
        fig: figura de plotly
        X: (n_samples, d) datos de entrada
        gmm: GaussianMixture ajustada
        pca: objeto PCA para (des)transformar las medias de las componentes Gaussianas y
            poder visualizarlas como imágenes.
        c_per_component: colores para cada componente Gaussiana. Admite arrays (ncomps, 3)
            con valores RGB en [0, 1]. Si es None, se mapearán las medias a colores RGB.
            show_covs: si True, muestra las elipsoides de covarianza.
        std_factors: factores, sobre la desviación estándar de cada componente Gaussiana,
            para controlar el tamaño de las elipsoides visualizadas.
        pca3_layout: si True, se ajusta el layout para visualizaciones 3D en espacio PCA.
        means_to_show: número de medias de las componentes Gaussianas a visualizar en 2D,
            como pseudo-leyenda.
        row, col: en caso de que la figura sea un subplot, determinan la posición del plot
            en la correspondiente cuadrícula.
    """
    means = gmm.means_
    ncomps = means.shape[0]
    covs = gmm.covariances_
    means3 = means[:, :3]

    if c_per_component is None:
        # map 3D points to RGB colors
        c_per_component = (means3 - means3.min(axis=0)) / np.ptp(means3, axis=0)

    if show_covs:
        # transforma las covarianzas a shape (ncomp, 3, 3)
        if gmm.covariance_type == "spherical":
            covs = np.eye(3)[None] * covs[:, None, None]
        elif gmm.covariance_type == "diag":
            covs = covs[:, :3]
            covs = np.stack([np.diag(cov) for cov in covs])
        elif gmm.covariance_type == "tied":
            covs = covs[:3, :3]
            covs = np.stack([covs] * ncomps)
        elif gmm.covariance_type == "full":
            covs = covs[:, :3, :3]
        else:
            raise ValueError

        plotly_add_ellipsoids(
            fig, covs, means3, c_per_component, std_factors, row=row, col=col
        )

    labels = gmm.predict(X)
    for i, (mean, color) in enumerate(zip(means, c_per_component)):
        X_comp = X[labels == i]
        m_cfg_point = {
            "color": np.repeat(color[None], X_comp.shape[0], axis=0),
            "size": 1.0,
            # "opacity": 0.1,
            "line_width": None,
        }
        m_cfg_means = {"size": 8.0, "symbol": "circle"}
        plotly_3d_points(fig, X_comp[:, :3], row=row, col=col, marker_cfg=m_cfg_point)
        plotly_3d_points(fig, mean[None], row=row, col=col, marker_cfg=m_cfg_means)

    if pca3_layout:
        # modificamos el layout, a uno apropiado para visualizaciones 3D en espacio PCA
        fig.update_layout(
            margin=(
                dict(r=0, l=0) if "scene2" in fig.layout else dict(r=0, b=10, l=0, t=10)
            ),
            hovermode=False,
            plot_bgcolor="white",
        )
        axis_kwargs = dict(showspikes=False, backgroundcolor="white", gridcolor="gray")
        fig.update_scenes(
            xaxis=dict(title="PC1", **axis_kwargs),
            yaxis=dict(title="PC2", **axis_kwargs),
            zaxis=dict(title="PC3", **axis_kwargs),
            # aspectmode="cube",
        )

    # visualización de (means_to_show) medias de las componentes Gaussianas como imágenes
    if pca is not None:
        subset = min(means_to_show, ncomps)
        means_espacio_imagen = pca.inverse_transform(means[:subset]).clip(0, 1)
        fig_, axs = visualize_subset_in_grid(
            means_espacio_imagen,
            subset=subset,
            random=False,
            ncols=subset,
            figsize=(3 * subset, 3),
        )
        # color del frame de cada subplot acorde a la visualización 3D anterior
        for ax, color, mean in zip(
            axs.ravel(), c_per_component[:subset], means_espacio_imagen
        ):
            for spine in ax.spines.values():
                spine.set_edgecolor(color)
                spine.set_linewidth(6)
        fig_.tight_layout()
        plt.show()

    fig.show()

## 2. GMM como proceso generativo

Una mezcla de Gaussianas (*GMM*) no solo sirve para clusterizar datos. En realidad, GMM, es un <br>
*estimador de densidad*, y por tanto podemos emplearlo para estimar o aproximar la distribución de <br>
probabilidad correspondiente a nuestros datos.

En esta parte de la práctica, vamos a hacer esto mismo sobre las imágenes (datos) de Fashion MNIST <br>
que empleásteis en la práctica anterior. Es decir, vuestro objetivo es estimar una distribución (GMM) <br>
que aproxime/explique las distribución real (desconocida) correspondiente a las imágenes de Fashion MNIST.

Por último, gracias a esta distribución estimada, vamos a poder generar nuevas imágenes, correspondientes <br>
a muestras de esta distribución que, hasta ese momento, no existían. <br>
En otras palabras: vamos a emplear un GMM como un *proceso generativo* de imágenes.

In [5]:
# cargamos el dataset
fashion_mnist = fetch_openml("Fashion-MNIST", parser="pandas")
X = fashion_mnist.data.values / 255  # (70000, 784=28x28) en [0, 1]
y = fashion_mnist.target.values.astype(int)  # (70000,) en [0, 9]
# label mapping
fmnist_labels = labels = {
    0: "Camiseta",
    1: "Pantalón",
    2: "Suéter",
    3: "Vestido",
    4: "Abrigo",
    5: "Sandalia",
    6: "Camisa",
    7: "Zapatilla",
    8: "Bolso",
    9: "Bota",
}
inv_labels = {v: k for k, v in labels.items()}

### 2. a) y b) Ajuste con GMM: BIC vs Nº de componentes Gaussianas

Decide cuántas componentes Gaussianas emplear teniendo en cuenta la métrica BIC. <br> 
Con este objetivo, completa la función `gmm_vs_gaussian_components`.

Importante: para acelerar los cálculos, emplearemos un subconjunto de los datos, así como reducir <br> 
su dimensionalidad con PCA, empleando para ello los argumentos  `subsample_size` y `pca_components`.


In [ ]:
def gmm_vs_gaussian_components(
    X: np.ndarray,
    gmm_components_to_try: np.ndarray,
    subsample_size: int = 5_000,
    pca_components: float | int = 0.95,
    seed: int = 0,
    verbose: bool = False,
    cov_type: str = "full",
):
    # reduce la cantidad de imágenes a emplear para acelerar cada fit de GMM
    rng = np.random.default_rng(seed)
    idx = rng.choice(X.shape[0], min(subsample_size, X.shape[0]), replace=False)
    X_sub = X[idx]
    if verbose:
        print(f"# imágenes originales: {X.shape[0]}")
        print(f"# imágenes seleccionadas aleatoriamente: {X_sub.shape[0]}")

    # reducción de dimensionalidad con PCA para (aún más) acelerar el fit de GMM acorde
    # a la cantidad de información (dimensiones) que estamos dispuestos a eliminar.
    pca = ...  # TODO
    X_pca = ...  # TODO
    if verbose:
        print(f"Reducción de dimensionalidad con PCA: {X.shape[1]} -> {X_pca.shape[1]}")

    # BIC vs Nº de componentes Gaussianas
    # TODO: tu código

    return X_pca, pca

Con la función anterior completada, y para obtener resultados lo más rápido posible, <br>
razona un número apropiado de componentes Gaussianas a utilizar para ajustar la distribución <br> 
de Fashion-MNIST, empleando:
1. Las 3 componentes más principales de los datos, 
2. Un subconjunto de 5000 imágenes, y
3. covarianzas de tipo `tied` 

In [ ]:
# TODO: tu código
# X_pca, pca = gmm_vs_gaussian_components(...)

### 2. c) Generación y visualización de imágenes

Ajusta un *GMM* con el número de componentes elegido en el apartado anterior y genera <br> 
nuevas imágenes mediante el sampleo/muestreo de puntos de esta distribución. Para visualizar <br> 
las imágenes muestreadas, recuerda que el GMM se ha ajustado en un espacio de dimensionalidad reducida.

Para su visualización, os recomendamos usar la función `visualize_subset_in_grid` definida <br>
al inicio del notebook.

In [ ]:
# TODO: tu código
# n_gaussian_components = ...
# gmm = GaussianMixture(...)
# ...

### 2. d) Genera y visualiza imágenes, empleando una única componente Gaussiana

Con el mismo GMM ajustado, elige una de sus componentes Gaussianas, y samplea/muestrea (y visualiza) <br> 
imágenes correspondientes a esta componente. ¿Qué observas? ¿El tipo de imágenes generadas es diferente?, ¿por qué?


In [ ]:
# TODO: tu código

### 2. e) Visualización 3D

Visualiza en 3D el GMM ajustado empleando la función `plotly_gmm_results_3d` definida al <br> 
inicio del notebook. ¿En qué espacio se está representando el ajuste?

In [ ]:
# TODO: tu código
# fig = go.Figure()
# plotly_gmm_results_3d(...)

### 2. f) Ajuste de GMM con un mayor (y apropiado) número de componentes principales

Por último, repite el proceso anterior--desde el apartado b) hasta el e), empleando un número <br> 
de componentes principales razonable para no perder tanta información. Siguiendo el mismo <br>
criterio de antes, ¿ha cambiado el número de componentes Gaussianas a emplear? ¿Por qué?

In [ ]:
# TODO: tu código
# X_pca, pca = gmm_vs_gaussian_components()

In [ ]:
# ajuste final
# TODO: tu código
# n_gaussian_components = ...
# gmm = GaussianMixture(...)

In [ ]:
# muestreo de imágenes
# TODO: tu código

In [ ]:
# muestreo de imágenes de una clase
# TODO: tu código

In [ ]:
# visualización 3D
# TODO: tu código
# fig = go.Figure()
# plotly_gmm_results_3d(...)

### 2. g) Opcional